# Part 1: Data Extraction

Author: Julianne <br>
Created: 1/28/2018<br>
Updated: 2/26/2018<br>

Step 1: Determine channels to use<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1) Extracted the top 50 most viewed category 26 videos of each month from the years 2010-2017. <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2) Determined unique channel names from previous list.<br>

Step 3: Extract videos from each channel<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3) Extracted all videos from the years 2010-2017 for each channel.<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;4) Shortened channel list to only include channels who have published at least 25 videos in catagory 26<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;5) Created final dataframe with all videos of the channels to be used.<br>

Step 4: Extract duration for each video <br>

Step 5: Extract channel information for each video<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1) Extract channel statistics<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2) Extract subscriber count, videoCount, viewCount<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3) Extract playlist information for each channel<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;4) Add channel and playlist information to main dataframe<br>

Step 6: Extract playlist information for each video

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext version_information
%version_information numpy, scipy, matplotlib, pandas

Software versions
Python 3.6.3 64bit [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 5.1.0
OS Darwin 16.1.0 x86_64 i386 64bit
numpy 1.13.3
scipy 0.18.1
matplotlib 2.0.0
pandas 0.19.2
Tue Feb 27 10:00:41 2018 EST

In [8]:
from __future__ import division
from datetime import datetime 
import requests
from lxml import html, etree
import json
from textblob import TextBlob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from collections import Counter
import timeit
from threading import Thread
import queue

#never print matching warnings
warnings.filterwarnings('ignore') 

#set how pandas displays data
pd.options.display.max_columns = 100
pd.options.display.max_rows = 35
pd.options.display.width = 120

#change the max # of rows pandas will display
pd.options.display.max_rows = 9000

#set to display all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#import seaborn to draw pretty graphs
import seaborn as sns
sns.set_style('whitegrid')

#name of files
name='Youtube_Part1_cat26_byview_2017_'

In [3]:
# API key
#api_key = ''
api_key = ''
#api_key = ''
#api_key = '' 
#api_key = '' 
#api_key = '' 
#api_key = '' 
#api_key = '' 
#api_key = '' 

# Step 1: Determine channels to use

## 1) Determine top 50 most viewed videos of catagory 26 for each month from the years 2010-2018

Define functions to be used

In [4]:
def find_videos_cat(publishedAfter, publishedBefore, maxResults, catId):
    parameters = {'part': "snippet",
                  'type': 'video',
                  'videoCategoryId': catId,
                  'order': 'viewCount',
                  'maxResults': maxResults,
                  'publishedAfter':publishedAfter,
                  'publishedBefore':publishedBefore,
                  'key': api_key,
                  }
    url = "https://www.googleapis.com/youtube/v3/search"
    page = requests.request(method="get", url=url, params=parameters)
    j_results = json.loads(page.text)

    snippet_id=[item["snippet"]["channelId"] for item in j_results["items"]]
    snippet_channel=[item["snippet"]["channelTitle"] for item in j_results["items"]]
    id_kind=[item["id"]["kind"] for item in j_results["items"]]
    id_videoid=[item["id"]["videoId"] for item in j_results["items"]]
    snippet_title=[item["snippet"]["title"] for item in j_results["items"]]
    snippet_description=[item["snippet"]["description"] for item in j_results["items"]]
    snippet_published=[item["snippet"]["publishedAt"] for item in j_results["items"]]

    col=['dateDownloaded','videoCategoryId','channelId','channelTitle','kind','videoId','title','description','publishedAt']
    rows=len(snippet_id)
    curr_data=pd.DataFrame(columns=col, index=range(0,rows))
    curr_data['dateDownloaded']=pd.Timestamp("today").strftime("%Y-%m-%dT%H:%M:%S")
    curr_data['videoCategoryId']=catId
    curr_data['channelId']=snippet_id
    curr_data['channelTitle']=snippet_channel
    curr_data['kind']=id_kind
    curr_data['videoId']=id_videoid
    curr_data['title']=snippet_title
    curr_data['description']=snippet_description
    curr_data['publishedAt']=snippet_published
    
    return curr_data

def get_data_YYYY_MM_begin(year, month, maxResults, catId):
    start=str(year)+"-"+str(month)+"-01T00:00:00Z"
    end=str(year)+"-"+str(month)+"-15T00:00:00Z"
    return find_videos_cat(publishedAfter=start, publishedBefore=end, maxResults=maxResults, catId=catId)

def get_data_YYYY_MM_end(year, month, maxResults, catId):
    date='30'
    if month=='01' or month=='05' or month=='07' or month=='08' or month=='10' or month=='12':
        date='31'
    if month=='02':      
        date='28'
    start=str(year)+"-"+str(month)+"-16T00:00:00Z"
    end=str(year)+"-"+str(month)+"-"+str(date)+"T00:00:00Z"
    return find_videos_cat(publishedAfter=start, publishedBefore=end, maxResults=maxResults, catId=catId)

Run code to extract videos using YouTube's API

In [8]:
maxResults=50
catId=26 #category Id, 26 is for lifestyle and beauty
years=range(2012,2018)
months=['01','02','03','04','05','06','07','08','09','10','11','12']

data=get_data_YYYY_MM_begin('2017', '01', 1, 20)

#create list of tasks to be done
data_queue = queue.Queue()
for year in years:
    for month in months:
        data_queue.put([year, month])
        
# create queues to save results
results = queue.Queue()
results_empty = queue.Queue()

def threadfnc(maxResults, catId):
    while not data_queue.empty():
        
        [year, month] = data_queue.get()
        curr_begin=get_data_YYYY_MM_begin(year, month, maxResults, catId)
        curr_end=get_data_YYYY_MM_begin(year, month, maxResults, catId)
        
        if curr_begin.empty:
            print(year, month, 'empty')
        else:
            results.put(curr_begin)
        if curr_end.empty:
            print(year, month, 'empty')
        else:
            results.put(curr_end) 
            
        data_queue.task_done()
    
for i in range(20):
    t = Thread(target=threadfnc, args=(maxResults, catId))
    t.start()

data_queue.join()
    
while not results.empty():
    curr = results.get()
    data = data.append(curr, ignore_index=True)
    results.task_done()
    
data.shape[0]
data.head(2)

Save data

In [5]:
#save as csv
filename=str(name)+'step1.csv'
#data.to_csv(filename)
df=pd.read_csv(filename)
del df['Unnamed: 0']
df.head(1)

,dateDownloaded,videoCategoryId,channelId,channelTitle,kind,videoId,title,description,publishedAt
0,2018-02-17T15:31:48,20,UCGIY_O-8vW4rfX98KlMkvRg,Nintendo,youtube#video,5kcdRBHM7kM,Super Mario Odyssey - Nintendo Switch Presenta...,Super Mario Odyssey sees Mario leave the Mushr...,2017-01-13T05:11:18.000Z


## 2) Extract information on all videos from specified channels

Determine names of channels that had at least one video in at least one month in the top 50 videos.

In [6]:
col=['channelId','frequency']
channel_list=pd.DataFrame(df.channelId.value_counts()).reset_index()
channel_list.columns=col
channel_list=channel_list[channel_list['frequency']>=1]
channel_list=np.array(channel_list)
len(channel_list)

1624

Next I will extract all videos from all channels of the previous list. Because API has a cap on how much data can be requested, the request must be split into multiple lists. Will use a different API for each list.

In [7]:
channel_list1=channel_list[0:300]
channel_list2=channel_list[300:600]
channel_list3=channel_list[600:900]
channel_list4=channel_list[900:1200]
channel_list5=channel_list[1200:1500]

Define functions to be used.

In [8]:
def find_videos(channelId, publishedAfter, publishedBefore, maxResults, vidfrequency, vidtype=26):
    parameters = {'part': 'snippet',
                  'channelId': channelId,
                  'type': "video",
                  'maxResults': maxResults,
                  'order': 'date',
                  'publishedAfter':publishedAfter,
                  'publishedBefore':publishedBefore,
                  'key': api_key,
                  }
    url = "https://www.googleapis.com/youtube/v3/search"
    col=['dateDownloaded','channelId','channelTitle','kind','videoId','title','description','publishedAt','vidfrequency','vidtype']
    curr_data=pd.DataFrame(columns=col)
    try:
        page = requests.request(method="get", url=url, params=parameters)
        j_results = json.loads(page.text)
        items = j_results.get("items", None)
        if items is None:
            return curr_data
        else:
            
            snippet_id=[item["snippet"]["channelId"] for item in j_results["items"]]
            snippet_channel=[item["snippet"]["channelTitle"] for item in j_results["items"]]
            id_kind=[item["id"]["kind"] for item in j_results["items"]]
            id_videoid=[item["id"]["videoId"] for item in j_results["items"]]
            snippet_title=[item["snippet"]["title"] for item in j_results["items"]]
            snippet_description=[item["snippet"]["description"] for item in j_results["items"]]
            snippet_published=[item["snippet"]["publishedAt"] for item in j_results["items"]]
            
            curr_data['channelId']=snippet_id
            curr_data['channelTitle']=snippet_channel
            curr_data['kind']=id_kind
            curr_data['videoId']=id_videoid
            curr_data['title']=snippet_title
            curr_data['description']=snippet_description
            curr_data['publishedAt']=snippet_published
            curr_data['vidfrequency']=vidfrequency
            curr_data['vidtype']=vidtype
            curr_data['dateDownloaded']=pd.Timestamp("today").strftime("%Y-%m-%dT%H:%M:%S")
            
            return curr_data
        
    except requests.exceptions.RequestException as e: 
        
        return curr_data

def get_data_YYYYbegin(channel_info, year, maxResults, vidtype):
    channelId=channel_info[0]
    vidfrequency=channel_info[1]
    start=str(year)+"-01-01T00:00:00Z"
    end=str(year)+"-6-30T00:00:00Z"
    return find_videos(channelId=channelId, publishedAfter=start, publishedBefore=end, maxResults=maxResults, vidfrequency=vidfrequency, vidtype=vidtype)

def get_data_YYYYend(channel_info, year, maxResults, vidtype):
    channelId=channel_info[0]
    vidfrequency=channel_info[1]
    start=str(year)+"-07-01T00:00:00Z"
    end=str(year)+"-12-31T00:00:00Z"
    return find_videos(channelId=channelId, publishedAfter=start, publishedBefore=end, maxResults=maxResults, vidfrequency=vidfrequency, vidtype=vidtype)

In [9]:
def extract_allvids(channel_list):
    
    maxResults=50
    vidtype=26
    years=range(2010,2018)
    
    data=get_data_YYYYbegin(channel_list[0], 2005, maxResults, vidtype)

    #create list of all tasks to be done
    data_queue = queue.Queue()
    for channel_info in channel_list:
        for year in years:
            data_queue.put((channel_info, year))

    # create queues to save results
    results = queue.Queue()

    def threadfnc(maxResults, vidtype):
        while not data_queue.empty():
            [channel_info, year] = data_queue.get()

            #get data from beginning of year
            curr_data=get_data_YYYYbegin(channel_info, year, maxResults, vidtype)
            if curr_data.empty:
                hi=5
            else:
                results.put(curr_data)

            #get data from end of year
            curr_data=get_data_YYYYend(channel_info, year, maxResults, vidtype)
            if curr_data.empty:
                hi=5
            else:
                results.put(curr_data) 
            data_queue.task_done()

    for i in range(20):
        t = Thread(target=threadfnc, args=(maxResults, vidtype))
        t.start()

    print('Step1: Extracting data')
    
    data_queue.join()
    
    print('Step2: Placing data in dataframe')

    while not results.empty():
        curr = results.get()
        data = data.append(curr, ignore_index=True)
        results.task_done()

    print('Done!')
        
    return data

Extract data for the specified channels. Use a different api key for each function.

In [13]:
tmp1=extract_allvids(channel_list1)

Step1: Extracting data
Step2: Placing data in dataframe
Done!


In [15]:
tmp2=extract_allvids(channel_list2)

Step1: Extracting data
Step2: Placing data in dataframe
Done!


In [17]:
tmp3=extract_allvids(channel_list3)

Step1: Extracting data
Step2: Placing data in dataframe
Done!


In [19]:
tmp4=extract_allvids(channel_list4)

Step1: Extracting data
Step2: Placing data in dataframe
Done!


merge the dataframes together

In [20]:
tmp1.shape[0]
tmp2.shape[0]
tmp3.shape[0]
tmp4.shape[0]

data=tmp1.append(tmp2).reset_index(drop=True)
data=data.append(tmp3).reset_index(drop=True)
data=data.append(tmp4).reset_index(drop=True)
data.shape[0]

102441

73677

62772

59177

298067

In [4]:
#save as csv
filename=str(name)+'step2.csv'
#data.to_csv(filename)
data=pd.read_csv(filename)
del data['Unnamed: 0']
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype
0,2018-02-26T12:48:50,UCGwPbAQdGA3_88WBuGtg9tw,grav3yardgirl,youtube#video,JrzIwYXX4lw,GORILLA GYM - DOES THIS THING REALLY WORK?,NEW SWAMP FAMILY STORE- http://www.swampmerch....,2015-06-27T20:03:01.000Z,117,26
1,2018-02-26T12:48:50,UCGwPbAQdGA3_88WBuGtg9tw,grav3yardgirl,youtube#video,JEI0m85O1gE,BLOTTERAZZI - FIRST IMPRESSION FRIDAY,NEW SWAMP FAMILY STORE- http://www.swampmerch....,2015-06-26T22:18:37.000Z,117,26


## 3) Extract catagory ID for each video

In [5]:
def get_video_snippet(video_id):
    parameters = {"part": "snippet",
              "id": video_id,
              "key": api_key
              }
    url = "https://www.googleapis.com/youtube/v3/videos"
    col=['videoId','categoryId']
    df=pd.DataFrame(columns=col,index=range(1))
    try:
        page = requests.request(method="get", url=url, params=parameters)
        j_results = json.loads(page.text)
        items = j_results.get("items", None)
        if items is None:
            return df
        else:
            if len(items)>0:
                df['videoId']=video_id
                sublist=items[0]
                if 'categoryId' in sublist['snippet']:
                    df['categoryId']=[item["snippet"]["categoryId"] for item in j_results["items"]][0]
                return df
            else:
                return df
    except requests.exceptions.RequestException as e: 
        return df

In [6]:
# data to work on
data=data

#create queue of all tasks to be done
data_queue = queue.Queue()
curr_list=list(data['videoId'])
for i in range(len(curr_list)):
    data_queue.put(curr_list[i])
    
# create queues to save results
results = queue.Queue()

def threadfnc():
    while not data_queue.empty():
        video_id = data_queue.get()
        df=get_video_snippet(video_id)
        results.put(df)
        data_queue.task_done()
        
for i in range(50):
    t = Thread(target=threadfnc, args=())
    t.start()

data_queue.join()

col=['videoId','categoryId']
df=pd.DataFrame(columns=col,index=range(0))

while not results.empty():
    curr = results.get()
    df = df.append(curr, ignore_index=True)
    results.task_done()

df.head(2)
    
data = pd.merge(data, df, on=['videoId'])
data.head(2)

,videoId,categoryId
0,JrzIwYXX4lw,26
1,JEI0m85O1gE,26


,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId
0,2018-02-26T12:48:50,UCGwPbAQdGA3_88WBuGtg9tw,grav3yardgirl,youtube#video,JrzIwYXX4lw,GORILLA GYM - DOES THIS THING REALLY WORK?,NEW SWAMP FAMILY STORE- http://www.swampmerch....,2015-06-27T20:03:01.000Z,117,26,26
1,2018-02-26T12:48:50,UCGwPbAQdGA3_88WBuGtg9tw,grav3yardgirl,youtube#video,JEI0m85O1gE,BLOTTERAZZI - FIRST IMPRESSION FRIDAY,NEW SWAMP FAMILY STORE- http://www.swampmerch....,2015-06-26T22:18:37.000Z,117,26,26


In [8]:
### save as csv
filename=str(name)+'step3.csv'
data.to_csv(filename)
data=pd.read_csv(filename)
del data['Unnamed: 0']
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId
0,2018-02-26T12:48:50,UCGwPbAQdGA3_88WBuGtg9tw,grav3yardgirl,youtube#video,JrzIwYXX4lw,GORILLA GYM - DOES THIS THING REALLY WORK?,NEW SWAMP FAMILY STORE- http://www.swampmerch....,2015-06-27T20:03:01.000Z,117,26,26
1,2018-02-26T12:48:50,UCGwPbAQdGA3_88WBuGtg9tw,grav3yardgirl,youtube#video,JEI0m85O1gE,BLOTTERAZZI - FIRST IMPRESSION FRIDAY,NEW SWAMP FAMILY STORE- http://www.swampmerch....,2015-06-26T22:18:37.000Z,117,26,26


## 4) Drop channels with fewer than 25 videos in catagory 26
Because of limits with the YouTube API, I cannot keep all channels. By dropping channels with fewer than 25 vidoes in catagory 26, I will have a small enough dataset to extract channel information more easily.

In [9]:
channels=pd.DataFrame(data.groupby(['channelTitle', 'categoryId']).agg('count')).reset_index()
channels=channels[channels['categoryId']==26].reset_index()
channels=channels[channels['dateDownloaded']>25].reset_index()
channels=channels[0:700]
channels['meetscriteria']='yes'
channels

,level_0,index,channelTitle,categoryId,dateDownloaded,channelId,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,meetscriteria
0,0,3,(Accidentally) Vegan TV,26,99,99,99,99,99,99,99,99,99,yes
1,1,10,*Owen Video,26,113,113,113,113,113,109,113,113,113,yes
2,2,13,10 Republic,26,50,50,50,50,50,50,50,50,50,yes
3,3,14,3MinuteTV,26,67,67,67,67,67,67,67,67,67,yes
4,4,15,5-Minute Crafts,26,150,150,150,150,150,150,150,150,150,yes
5,5,16,5-Minute Crafts KIDS,26,50,50,50,50,50,50,50,50,50,yes
6,7,21,ACADEMIA DE MUSICA SIERREÑA,26,93,93,93,93,93,80,93,93,93,yes
7,8,24,AFP news agency,26,79,79,79,79,79,79,79,79,79,yes
8,9,29,ATHLEAN-X™,26,624,624,624,624,624,624,624,624,624,yes
9,11,34,Aayiye Sikhte Hai,26,84,84,84,84,84,84,84,84,84,yes


In [10]:
channels=channels[['channelTitle','meetscriteria']]
channels.head(2)

,channelTitle,meetscriteria
0,(Accidentally) Vegan TV,yes
1,*Owen Video,yes


In [11]:
data = pd.merge(data, channels, how='left', on='channelTitle')
data = data[data['meetscriteria']=='yes']
del data['meetscriteria']
data.shape[0]
data.head(2)

194133

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId
196,2018-02-26T12:48:50,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26
197,2018-02-26T12:48:50,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26


In [5]:
#save as csv
filename=str(name)+'step4.csv'
data.to_csv(filename)
data=pd.read_csv(filename)
del data['Unnamed: 0']
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId
0,2018-02-26T12:48:50,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26
1,2018-02-26T12:48:50,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26


# Step 2: Extract duration for each video

In [6]:
def get_video_duration(video_id):
    parameters = {"part": "contentDetails",
              "id": video_id,
              "key": api_key
              }
    url = "https://www.googleapis.com/youtube/v3/videos"
    try:
        page = requests.request(method="get", url=url, params=parameters)
        j_results = json.loads(page.text)
        items = j_results.get("items", None)
        if items is None:
            return 'NA'
        else:
            if len(items)>0:
                sublist=items[0]
                if 'duration' in sublist['contentDetails']:
                    return [item["contentDetails"]["duration"] for item in j_results["items"]][0]
                else:
                    return 'NA'
            else:
                return 'NA'
    except requests.exceptions.RequestException as e: 
        return -1

In [7]:
#data to work on
data=data

#create queue of all tasks to be done
data_queue = queue.Queue()
curr_list=list(data['videoId'])
for i in range(len(curr_list)):
    data_queue.put(curr_list[i])
    
# create queues to save results
results = queue.Queue()

def threadfnc():
    while not data_queue.empty():
        video_id = data_queue.get()
        curr_data=get_video_duration(video_id)
        results.put([video_id,curr_data])
        data_queue.task_done()
        
for i in range(30):
    t = Thread(target=threadfnc, args=())
    t.start()

data_queue.join()

col=['videoId','videoDuration']
curr_data=pd.DataFrame(columns=col)
while not results.empty():
    curr = results.get()
    curr=pd.DataFrame([curr], columns=col)
    curr_data = curr_data.append(curr, ignore_index=True)
    results.task_done()

curr_data.head(2)
    
data = pd.merge(data, curr_data, on=['videoId'])
data.head(2)

,videoId,videoDuration
0,axhPJi626SM,PT5M19S
1,v4rNOLZgKF4,PT8M11S


,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration
0,2018-02-26T12:48:50,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S
1,2018-02-26T12:48:50,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S


In [4]:
#save as csv
filename=str(name)+'step5.csv'
data.to_csv(filename)
data=pd.read_csv(filename)
del data['Unnamed: 0']
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration
0,2018-02-26T12:48:50,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S
1,2018-02-26T12:48:50,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S


# Step 3: Add statistics for each video

In [5]:
def get_video_statistics(video_id):
    parameters = {"part": "statistics",
              "id": video_id,
              "key": api_key
              }
    url = "https://www.googleapis.com/youtube/v3/videos"
    col=['videoId','commentCount','dislikeCount','likeCount','viewCount']
    df=pd.DataFrame(columns=col,index=range(1))
    try:
        page = requests.request(method="get", url=url, params=parameters)
        j_results = json.loads(page.text)
        items = j_results.get("items", None)
        if items is None:
            return df
        else:
            if len(items)>0:
                df['videoId']=[item["id"] for item in j_results["items"]][0]
                sublist=items[0]
                if 'commentCount' in sublist['statistics']:
                    df['commentCount']=[item["statistics"]["commentCount"] for item in j_results["items"]][0]
                if 'dislikeCount' in sublist['statistics']:
                    df['dislikeCount']=[item["statistics"]["dislikeCount"] for item in j_results["items"]][0]
                if 'likeCount' in sublist['statistics']:
                    df['likeCount']=[item["statistics"]["likeCount"] for item in j_results["items"]][0]
                if 'viewCount' in sublist['statistics']:
                    df['viewCount']=[item["statistics"]["viewCount"] for item in j_results["items"]][0]
                return df
            else:
                return df
    except requests.exceptions.RequestException as e: 
        return df

In [10]:
# data to work on
data=data

#create queue of all tasks to be done
data_queue = queue.Queue()
curr_list=list(data['videoId'])
for i in range(len(curr_list)):
    data_queue.put(curr_list[i])
    
# create queues to save results
results = queue.Queue()

def threadfnc():
    while not data_queue.empty():
        video_id = data_queue.get()
        df=get_video_statistics(video_id)
        results.put(df)
        data_queue.task_done()
        
for i in range(20):
    t = Thread(target=threadfnc, args=())
    t.start()

print('Part 1')    

data_queue.join()

print('Part 2')

col=['videoId','commentCount','dislikeCount','likeCount','viewCount']
df=pd.DataFrame(columns=col,index=range(0))

while not results.empty():
    curr = results.get()
    df = df.append(curr, ignore_index=True)
    results.task_done()

df.head(2)
    
data = pd.merge(data, df, on=['videoId'])
data.head(2)

Part 1
Part 2


,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0


In [9]:
#save as csv
filename=str(name)+'step6.csv'
#data.to_csv(filename)
data=pd.read_csv(filename)
del data['Unnamed: 0']
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0


# Step 4: Extract channel-related information

## 1) Extract channel statistics

In [34]:
#create small dataframe with channel information
channels=data.channelId.unique()
col=['dateDownloaded','channelTitle','channelId']
rows=len(channels)
df_channels=pd.DataFrame(columns=col,index=range(0,rows))
for i in range(0,rows):
    df_channels['dateDownloaded'][i]=pd.Timestamp("today").strftime("%Y-%m-%dT%H:%M:%S")
    df_channels['channelTitle'][i]=np.array(data[data['channelId']==channels[i]].channelTitle)[0]
    df_channels['channelId'][i]=channels[i]
df_channels.head(2)

,dateDownloaded,channelTitle,channelId
0,2018-02-25T17:13:50,BRIGHT SIDE,UC4rlAVgAK0SGk-yTfe48Qpw
1,2018-02-25T17:13:50,ATHLEAN-X™,UCe0TLA0EsQbE-MjuHXevj2A


## 2) Extract subscriber count, videoCount, viewCount

In [37]:
df_channels['commentCount']='NA'
df_channels['hiddenSubscriberCount']='NA'
df_channels['subscriberCount']='NA'
df_channels['videoCount']='NA'
df_channels['viewCount']='NA'

for i in range(0,df_channels.shape[0]):
    curr_channel=df_channels.channelId[i]
    parameters = {"part": "statistics",
          "id": curr_channel,
          "key": api_key
          }
    url = "https://www.googleapis.com/youtube/v3/channels"
    page = requests.request(method="get", url=url, params=parameters)
    j_results = json.loads(page.text)
    df_channels["commentCount"][i] = [item["statistics"]["commentCount"] for item in j_results["items"]][0]
    df_channels["hiddenSubscriberCount"][i] = [item["statistics"]["hiddenSubscriberCount"] for item in j_results["items"]][0]
    df_channels["subscriberCount"][i] = [item["statistics"]["subscriberCount"] for item in j_results["items"]][0]
    df_channels["videoCount"][i] = [item["statistics"]["videoCount"] for item in j_results["items"]][0]
    df_channels["viewCount"][i] = [item["statistics"]["viewCount"] for item in j_results["items"]][0]
df_channels.head(2)

,dateDownloaded,channelTitle,channelId,commentCount,hiddenSubscriberCount,subscriberCount,videoCount,viewCount
0,2018-02-25T17:13:50,BRIGHT SIDE,UC4rlAVgAK0SGk-yTfe48Qpw,0,False,7860308,894,1523479095
1,2018-02-25T17:13:50,ATHLEAN-X™,UCe0TLA0EsQbE-MjuHXevj2A,3650,False,4612450,888,665880213


In [40]:
#save as csv
#df_channels.to_csv('Youtube_cat26_byDate_channelInfo.csv')
#df_channels=pd.read_csv('Youtube_cat26_byDate_channelInfo.csv')
#del df_channels['Unnamed: 0']
df_channels.head(2)

,dateDownloaded,channelTitle,channelId,commentCount,hiddenSubscriberCount,subscriberCount,videoCount,viewCount
0,2018-02-25T17:13:50,BRIGHT SIDE,UC4rlAVgAK0SGk-yTfe48Qpw,0,False,7860308,894,1523479095
1,2018-02-25T17:13:50,ATHLEAN-X™,UCe0TLA0EsQbE-MjuHXevj2A,3650,False,4612450,888,665880213


## 3) Extract playlist information for each channel

Extract a list of all playlistIds for each channel in df_channels.

In [41]:
col=['id', 'kind', 'channelId', 'channelTitle', 'description', 'title', 'publishedAt']
playlist_df=pd.DataFrame(columns=col)
for channel in list(df_channels.channelId):
    parameters = {"part": "snippet",
                  "channelId": channel,
                  'maxResults': 50,
                  "key": api_key
                  }
    url = "https://www.googleapis.com/youtube/v3/playlists"
    page = requests.request(method="get", url=url, params=parameters)
    j_results = json.loads(page.text)
    
    ids=[item["id"] for item in j_results["items"]]
    kinds=[item["kind"] for item in j_results["items"]]
    channelIds=[item["snippet"]['channelId'] for item in j_results["items"]]
    channelTitles=[item["snippet"]['channelTitle'] for item in j_results["items"]]
    desc=[item["snippet"]['description'] for item in j_results["items"]]
    title=[item["snippet"]['title'] for item in j_results["items"]]
    publishedAts=[item["snippet"]['publishedAt'] for item in j_results["items"]]
    
    curr_df=pd.DataFrame(columns=col, index=range(0,len(ids)))
    curr_df['id']=ids
    curr_df['kind']=kinds
    curr_df['channelId']=channelIds
    curr_df['channelTitle']=channelTitles
    curr_df['description']=desc
    curr_df['title']=title
    curr_df['publishedAt']=publishedAts
    
    playlist_df=playlist_df.append(curr_df, ignore_index=True) ## need to use ignore_index=True in order to make new index

playlist_df.head(2)

,id,kind,channelId,channelTitle,description,title,publishedAt
0,PLVNocu2A1SADjisuByJd3zA_EUA7cqKl-,youtube#playlist,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,,Life Hacks,2018-02-05T13:29:38.000Z
1,PLVNocu2A1SAAgwy9JSXM7k3IoKiuqsTQx,youtube#playlist,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,Amazing facts about human body you've probably...,Human Body,2018-01-21T10:38:49.000Z


Add the number of playlists for each channel in df_channels, 

In [42]:
# determine how many playlists per channel
df_channels['num_of_playlists']=0
row=-1
for i in range(df_channels.shape[0]):
    row=row+1
    channel=df_channels['channelTitle'][row]
    currdf=playlist_df[playlist_df['channelTitle']==channel]
    df_channels['num_of_playlists'][row]=currdf.shape[0]
df_channels.head(2)

,dateDownloaded,channelTitle,channelId,commentCount,hiddenSubscriberCount,subscriberCount,videoCount,viewCount,num_of_playlists
0,2018-02-25T17:13:50,BRIGHT SIDE,UC4rlAVgAK0SGk-yTfe48Qpw,0,False,7860308,894,1523479095,17
1,2018-02-25T17:13:50,ATHLEAN-X™,UCe0TLA0EsQbE-MjuHXevj2A,3650,False,4612450,888,665880213,21


In [44]:
#save as csv
playlist_df.to_csv('Youtube_Part1_cat26_byView_playlistInfo.csv')
playlist_df=pd.read_csv('Youtube_Part1_cat26_byView_playlistInfo.csv')
del playlist_df['Unnamed: 0']
playlist_df.head(2)

,id,kind,channelId,channelTitle,description,title,publishedAt
0,PLVNocu2A1SADjisuByJd3zA_EUA7cqKl-,youtube#playlist,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,,Life Hacks,2018-02-05T13:29:38.000Z
1,PLVNocu2A1SAAgwy9JSXM7k3IoKiuqsTQx,youtube#playlist,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,Amazing facts about human body you've probably...,Human Body,2018-01-21T10:38:49.000Z


## 4) Add channel and playlist information to main dataframe

In [45]:
#select columns
subdf=df_channels.loc[:,['channelId','commentCount','hiddenSubscriberCount','subscriberCount','videoCount','viewCount','num_of_playlists']]

#rename columns
subdf.columns=['channelId','stats_commentCount','stats_hiddenSubscriberCount','stats_subscriberCount','stats_videoCount','stats_viewCount','num_of_playlists']
subdf.head(2)

#merge dataframes
data = pd.merge(data, subdf, on=['channelId'])
data.head(2)

,channelId,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists
0,UC4rlAVgAK0SGk-yTfe48Qpw,0,False,7860308,894,1523479095,17
1,UCe0TLA0EsQbE-MjuHXevj2A,3650,False,4612450,888,665880213,21


,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7860308,894,1523479095,17
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7860308,894,1523479095,17


In [46]:
#save as csv
filename=str(name)+'step10.csv'
data.to_csv(filename)
data=pd.read_csv(filename)
del data['Unnamed: 0']
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7860308,894,1523479095,17
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7860308,894,1523479095,17


# Step 5: Determine if video is in a playlist or not

Create dataframe with all videos in the playlists

In [47]:
def get_video_playlist(playlist):
    
    parameters = {"part": "snippet",
                  "playlistId": playlist,
                  'maxResults': 50,
                  "key": api_key
                  }
    url = "https://www.googleapis.com/youtube/v3/playlistItems"
    col=['playlistId', 'position' , 'videoId']
    df=pd.DataFrame(columns=col)
    try:
        page = requests.request(method="get", url=url, params=parameters)
        j_results = json.loads(page.text)
        items = j_results.get("items", None)
        if items is None:
            return df
        else: 
            if len(items)>0:
                sublist=items[0]   
                if 'playlistId' in sublist['snippet']:
                    df['playlistId']=[item["snippet"]["playlistId"] for item in j_results["items"]]
                if 'position' in sublist['snippet']:
                    df['position']=[item["snippet"]["position"] for item in j_results["items"]]
                    df['videoId']=[item["snippet"]["resourceId"]['videoId'] for item in j_results["items"]]
                return df
            else:
                return df
    except requests.exceptions.RequestException as e: 
        return df   

In [48]:
# data to work on
playlist_df=playlist_df

#create queue of all tasks to be done
data_queue = queue.Queue()
curr_list=list(playlist_df['id'])
for i in range(len(curr_list)):
    data_queue.put(curr_list[i])
    
# create queues to save results
results = queue.Queue()

def threadfnc():
    while not data_queue.empty():
        playlist = data_queue.get()
        df=get_video_playlist(playlist)
        results.put(df)
        data_queue.task_done()
        
for i in range(100):
    t = Thread(target=threadfnc, args=())
    t.start()

data_queue.join()

col=['playlistId', 'position' , 'videoId']
df=pd.DataFrame(columns=col,index=range(0))

while not results.empty():
    curr = results.get()
    df = df.append(curr, ignore_index=True)
    results.task_done()

df.head(2)

,playlistId,position,videoId
0,PLVNocu2A1SADjisuByJd3zA_EUA7cqKl-,0.0,gUppezqI3yc
1,PLVNocu2A1SADjisuByJd3zA_EUA7cqKl-,1.0,AVSCjxSqK2Q


Count number of playlists that each video was included in

In [49]:
col=['videoId','num_of_playlists_forvid']
videoId_list=pd.DataFrame(df.videoId.value_counts()).reset_index()
videoId_list.columns=col
videoId_list

,videoId,num_of_playlists_forvid
0,wtPvhIocZxs,40
1,ZlTRurKVYJE,39
2,IMNOnqUm_Jc,39
3,5QttvrmtcfQ,38
4,xomVeuB2Ld4,31
5,huX0Dn3RZJE,28
6,Vuxnvk96Igg,22
7,148n-j5MBPg,22
8,eDaLxaeArt4,21
9,KKydUVdsuLI,21


Merge freqeuncy of vid in playlist into main dataframe

In [50]:
data = pd.merge(data, videoId_list, how='left', on='videoId')
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7860308,894,1523479095,17,1.0
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7860308,894,1523479095,17,NaN


In [53]:
#save as csv
filename=str(name)+'_Final.csv'
data.to_csv(filename)
data=pd.read_csv(filename)
del data['Unnamed: 0']
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7860308,894,1523479095,17,1.0
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7860308,894,1523479095,17,NaN
